In [1]:
%%capture
import warnings
warnings.filterwarnings("ignore")

!pip install langchain langchain-community openai requests gradio --quiet

In [2]:
import os
import requests
import pandas as pd
from langchain.chat_models import ChatOpenAI
import gradio as gr
from langchain.tools import tool
from langchain.agents import initialize_agent

In [3]:
#API keys
os.environ["OPENAI_API_KEY"] = "ENTER YOUR OPENAI KEY HERE"
NUTRITIONIX_APP_ID = "ENTER YOUR NUTRITIONIX_API_ID HERE"
NUTRITIONIX_API_KEY = "ENTER YOUR NUTRITIONIX_API_KEY HERE"
USDA_API_KEY = "ENTER YOUR USDA_API_KEY HERE"
USDA_API_URL = "ENTER YOUR USDA_API_URL HERE"

In [4]:
# Initialize calorie log as a Pandas DataFrame
calorie_log = pd.DataFrame(columns=["Date", "Meal", "Food", "Calories"])

# Initialize LangChain ChatOpenAI model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)


C:\Users\Rohit Chowdary\AppData\Local\Temp\ipykernel_25332\1505246425.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)


In [5]:
@tool
def fetch_calories_tool(food_item: str) -> str:
    """Fetch calorie information for a given food item using the Nutritionix API."""
    url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
    headers = {
        "x-app-id": NUTRITIONIX_APP_ID,
        "x-app-key": NUTRITIONIX_API_KEY,
        "Content-Type": "application/json"
    }
    data = {"query": food_item}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        if result.get("foods"):
            calories = result["foods"][0]["nf_calories"]
            return f"The calorie content of {food_item} is {calories} kcal."
    return f"Could not fetch calorie information for {food_item}."

In [6]:
@tool
def get_major_nutrients_tool(food_item: str) -> str:
    """Fetch major nutrient data (calories, protein, carbs, fat) using the USDA API."""
    params = {"query": food_item, "api_key": USDA_API_KEY}
    response = requests.get(USDA_API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["foods"]:
            food = data["foods"][0]
            nutrients = {n["nutrientName"]: n["value"] for n in food["foodNutrients"]}
            major_nutrients = {
                "Calories": nutrients.get("Energy (kcal)", "N/A"),
                "Protein": nutrients.get("Protein", "N/A"),
                "Carbohydrates": nutrients.get("Carbohydrate, by difference", "N/A"),
                "Fat": nutrients.get("Total lipid (fat)", "N/A")
            }
            return (
                f"Major nutrients for {food['description']}:\n"
                f"- Protein: {major_nutrients['Protein']} g\n"
                f"- Carbs: {major_nutrients['Carbohydrates']} g\n"
                f"- Fat: {major_nutrients['Fat']} g"
            )
    return f"Could not fetch nutrient information for {food_item}."


In [7]:
def suggest_meal_with_nutrients_tool(remaining_calories: str) -> str:
    """
    Suggest meals/snacks under the given calorie input and provide short nutrient details.
    """
    try:
        remaining_calories = float(remaining_calories)
    except ValueError:
        return "Invalid input. Please provide a numeric value for remaining calories."
    
    # Using LLM to generate meal suggestions
    prompt = f"Suggest three healthy meals/snacks under {remaining_calories} calories. List them in bullet points."
    meal_suggestions = llm.predict(prompt)
    meal_suggestions_list = [meal.strip("- ").strip() for meal in meal_suggestions.split("\n") if meal.startswith("-")]
    
    if not meal_suggestions_list:
        return "No valid meal suggestions generated."
    
    # Start building response
    response = f"Calorie Input Provided: {remaining_calories} kcal\n\n"
    response += "Meal suggestions with nutrient details:\n"
    
    # Fetch nutrient details for each suggestion using the tool
    for meal in meal_suggestions_list:
        nutrient_info = get_major_nutrients_tool(meal)
        response += f"- {nutrient_info}\n\n"
    return response


In [8]:
# Initialize LangChain agent
chat_model = ChatOpenAI(temperature=0.3)
tools = [fetch_calories_tool, get_major_nutrients_tool]
agent = initialize_agent(tools, chat_model, agent="zero-shot-react-description", handle_parsing_errors=True)

C:\Users\Rohit Chowdary\AppData\Local\Temp\ipykernel_25332\3068156768.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(tools, chat_model, agent="zero-shot-react-description", handle_parsing_errors=True)


In [10]:
# Updated Functions Using Agent
def log_meal_gradio(food_item, meal_type):
    """Handles meal logging via the agent."""
    global calorie_log
    
    if not food_item or not meal_type:
        return "Please provide both a food item and a meal type."
    
    # Query the agent for calorie information
    query = f"Fetch the calorie content of {food_item}."
    try:
        calories_info = agent.run(query)
        calories = float(calories_info.split("is")[1].split("kcal")[0].strip())  # Extract calorie value
    except (IndexError, ValueError):
        return "Could not fetch valid calorie information for the given food item."
    
    # Log the meal
    new_entry = pd.DataFrame([{
        "Date": pd.Timestamp.now().date(),
        "Meal": meal_type,
        "Food": food_item,
        "Calories": calories
    }])
    calorie_log = pd.concat([calorie_log, new_entry], ignore_index=True)
    
    return f"Logged: {food_item} ({meal_type}) with {calories} kcal."

In [11]:
def get_calorie_summary_gradio():
    """Provides a summary of the logged meals."""
    global calorie_log
    
    if calorie_log.empty:
        return "No meals have been logged yet."
    
    # Generate summary
    total_calories = calorie_log["Calories"].sum()
    summary = calorie_log.groupby(["Meal"])["Calories"].sum().reset_index()
    summary_text = "Calorie Summary:\n"
    for _, row in summary.iterrows():
        summary_text += f"- {row['Meal']}: {row['Calories']} kcal\n"
    
    summary_text += f"Total Calories: {total_calories} kcal"
    return summary_text

In [16]:
# Gradio interface remains the same
with gr.Blocks() as demo:
    gr.Markdown("# 🥗 Calorie Tracker and Nutrient Assistant")
    gr.Markdown("Log your meals, get calorie summaries, or receive meal and nutrient suggestions based on your calorie limits.")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Log a Meal")
            food_input = gr.Textbox(label="Food Item", placeholder="E.g., Pasta")
            meal_type_dropdown = gr.Dropdown(["Breakfast", "Lunch", "Dinner", "Snack"], label="Meal Type", value="Lunch")
            log_meal_button = gr.Button("Log Meal")
        
        with gr.Column():
            gr.Markdown("### Get Calorie Summary")
            get_summary_button = gr.Button("Get Summary")
        
        with gr.Column():
            gr.Markdown("### Suggest a Meal with Nutrients")
            remaining_calories_input = gr.Textbox(label="Remaining Calories", placeholder="Enter the calories E.g., 500")
            suggest_meal_button = gr.Button("Suggest Meal")
    
    response_output = gr.Textbox(label="Response", interactive=False)
    
    log_meal_button.click(log_meal_gradio, inputs=[food_input, meal_type_dropdown], outputs=response_output)
    get_summary_button.click(get_calorie_summary_gradio, inputs=None, outputs=response_output)
    suggest_meal_button.click(suggest_meal_with_nutrients_tool, inputs=remaining_calories_input, outputs=response_output)

if __name__ == "__main__":
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://7817f7c3a730d4ffa8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
